In [807]:
# linux
username = "nova"
pathtorepo = "/home/nova/work/repo_ds/dynamical-systems"
using Pkg
Pkg.activate(pathtorepo * "/env/integrate/")

  Activating project at `~/work/repo_ds/dynamical-systems/env/integrate`


In [808]:
"""# windows
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")"""

"# windows\nusername = \"Alex\"\npathtorepo = \"C:\\Users\\\" *username *  \"\\Desktop\\\"\nusing Pkg\nPkg.activate(pathtorepo * \"dynamical-systems\\env\\integrate\\\")"

In [809]:
using StaticArrays, DifferentialEquations, DynamicalSystems, ForwardDiff, BenchmarkTools, IntervalRootFinding, LinearAlgebra
using CairoMakie, GLMakie

In [810]:
@inbounds function tetra(var, par, t)

    g = log( 1.0 + exp( ( (par[9] + par[6] * var[5]) * var[3] * var[2] * var[1] + par[11]) / par[5] ) );
    U = par[10] + par[12] / ( 1.0 + exp( -50.0 * ( var[4] - par[25] ) ) );
    Hy = 1.0 / ( 1.0 + exp( -20.0 * ( var[2] - par[26] ) ) );
    Hecm = par[17] - (par[17] - par[18]) / (1.0 + exp( -(var[1] - par[20]) / par[19] ) );
    Hp =  par[21] - (par[21] - par[22]) / (1.0 + exp( -(var[1] - par[23]) / par[24]) );

    dE = (-var[1] + par[5] * g) / par[1];
    dx = (1.0 - var[2]) / par[2]  -var[3] * var[2] * var[1];
    du = (U - var[3]) / par[3]  + U * (1.0 - var[3]) * var[1];
    dy = (-var[4]) / par[4] + par[13] * Hy;
    decm = -( par[7] + par[16] * var[6] ) * var[5] + par[14] * Hecm; 
    dp = -par[8] * var[6] + par[15] * Hp;

    return SVector(dE, dx, du, dy, decm, dp);
end

@inbounds function tetra_jac(vars, params, t)

    E_sum =  params[9] + params[6] * vars[5];

    exp_ = exp( ( E_sum * vars[3] * vars[2] * vars[1] + params[11] ) / params[5] );
    
    denomE = 1.0 + exp_;

    exp50 = exp( -50.0 * ( vars[4] - params[25] ) );
    exp20 = exp( -20.0 * ( vars[2] - params[26] ) );
    exphecm = exp( -(vars[1] - params[20]) / (params[19]) );
    exphp = exp( -(vars[1] - params[23]) / (params[24]) );

    U = params[10] + params[12] / ( 1.0 + exp( -50.0 * ( vars[4] - params[25] ) ) );
    Uder = (50.0 * params[12] * exp50) / ( 1.0 + exp50 )^2;

    Hyder = 20.0 * exp20 / (1.0 + exp20)^2;

    Hecmder = (params[17] - params[18]) * exphecm / ( params[19] * (1.0 + exphecm)^2 );
    Hpder = (params[21] - params[22]) * exphp / ( params[24] * (1.0 + exphp)^2 );

    EE = ( -1.0 + ( E_sum * vars[3] * vars[2] * exp_ ) / denomE ) / params[1];
    Ex = ( ( E_sum * vars[3] * vars[1] * exp_ ) / denomE ) / params[1];
    Eu = ( ( E_sum * vars[2] * vars[1] * exp_ ) / denomE ) / params[1];
    Eecm = ( (exp_ * params[6] * vars[3] * vars[2] * vars[1]) / denomE ) / params[1];

    xE = -vars[3] * vars[2];
    xx = - 1.0 / params[2] - vars[3] * vars[1];
    xu = -vars[2] * vars[1];


    uE = U * (1.0 - vars[3]);
    uu = -1.0 / params[3] - U * vars[1];
    uy = Uder * ( 1.0 / params[3] + (1.0 - vars[3]) * vars[1] );

    yx = params[13] * Hyder;

    ecmE =  -params[14] * Hecmder;
    ecmecm = -(params[7] + params[16] * vars[6]);
    ecmp = -vars[5] * params[16];

    pE = -params[15] * Hpder;

    SMatrix{6,6}(EE, xE, uE, 0.0, ecmE, pE,
        Ex, xx, 0.0, yx, 0.0, 0.0,
        Eu, xu, uu, 0.0, 0.0, 0.0,
        0.0, 0.0, uy, - 1.0 / params[4], 0.0, 0.0,
        Eecm, 0.0, 0.0, 0.0, ecmecm, 0.0,
        0.0, 0.0, 0.0, 0.0, ecmp, -params[8])
end

tetra_jac (generic function with 1 method)

In [811]:
time = 5000.0; tt = 0.0; tstep = 0.001; ttr = 2500.0; times = [time, tt];
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [812]:
τ = 0.05; τD = 0.15; τF = 1.0; τy = 3.3;   
α = 1.58; αecm = 0.001; αp = 0.01;
J = 3.07; U0 = 0.3; ΔU0 = 0.305; 
β = 0.438; βp = 0.001; βecm = 0.01;
ecm0 = 0.0; ecm1 = 1.0; kecm = 0.15; θecm = 30;
p0 = 0.0; p1 = 1.0; kp = 0.05; γp = 0.1; θp = 26.0; 
ythr = 0.4; xthr = 0.75;

In [813]:
u01 = [2.422092651924434, 0.8446031114662503, 0.6534291088738378, 0.09540062161211987, 0.0004056327238809937, 1.4805608752412292e-9];
u02 = ones(6);
u03 = [8.8746, 0.4815, 0.8089, 0.0, 0.0, 0.0];

In [814]:
# control parameter
αE = 5.23;
I0 = -1.741;
param = SA[τ, τD, τF, τy, α, αE, αecm, αp, J, U0, I0, ΔU0, β, βecm, βp, γp, ecm0, ecm1, kecm, θecm, p0, p1, θp, kp, ythr, xthr];
ds = CoupledODEs(tetra, u03, param, diffeq = integ_set);

In [815]:
Jacobi(u0) = ForwardDiff.jacobian((x) -> tetra(x, param, 0.0), u0);

In [816]:
jac_ana = Matrix(tetra_jac(u01, param, 0.0))

6×6 Matrix{Float64}:
  7.70926      79.4627    102.711     0.0         114.256   0.0
 -0.551888     -8.24933    -2.04571   0.0           0.0     0.0
  0.103971      0.0        -1.72663   6.81806e-6    0.0     0.0
  0.0           0.997291    0.0      -0.30303       0.0     0.0
  9.49924e-82   0.0         0.0       0.0          -0.001  -4.05633e-5
  0.0           0.0         0.0       0.0           0.0    -0.01

In [817]:
jac_num = Jacobi(u01)

6×6 Matrix{Float64}:
  7.70926      79.4627    102.711     0.0         114.256   0.0
 -0.551888     -8.24933    -2.04571  -0.0          -0.0    -0.0
  0.103971      0.0        -1.72663   6.81806e-6    0.0     0.0
  0.0           0.997291    0.0      -0.30303       0.0     0.0
  9.49924e-82   0.0         0.0       0.0          -0.001  -4.05633e-5
  0.0           0.0         0.0       0.0           0.0    -0.01

In [818]:
0.0004056327238809937 * γp

4.056327238809937e-5

In [819]:
row = 5
col = 5
jac_ana[row, :] == jac_num[row, :]

false

In [820]:
jac_ana[row, :]

6-element Vector{Float64}:
  9.499239764096717e-82
  0.0
  0.0
  0.0
 -0.0010000001480560876
 -4.056327238809937e-5

In [821]:
jac_num[row, :]

6-element Vector{Float64}:
  9.499239764096719e-82
  0.0
  0.0
  0.0
 -0.0010000001480560876
 -4.056327238809937e-5

In [822]:
Er = interval(0.0, 10.0);
xr, ur, yr = interval(0.0, 1.0), interval(0.0, 1.0), interval(0.0, 1.0);
ecmr, pr = interval(0.0, 0.1), interval(0.0, 0.1);
box = IntervalBox(Er, xr, ur, yr, ecmr, pr);

In [823]:
fp, eigs, stable = fixedpoints(ds, box, tetra_jac, tol = 1e-14, method = IntervalRootFinding.Newton)

┌ Warning: Non-unique fixed points found!
└ @ ChaosTools /home/nova/.julia/packages/ChaosTools/Nr2Oq/src/stability/fixedpoints.jl:103


(6-dimensional StateSpaceSet{Float64} with 6 points, Vector{ComplexF64}[[-5.2896408370108565 - 8.88992404577992im, -5.2896408370108565 + 8.88992404577992im, -3.1129277710557277 + 0.0im, -0.3030303044047327 + 0.0im, -0.01 + 0.0im, -0.001 + 0.0im], [-5.289640837010866 - 8.889924045779937im, -5.289640837010866 + 8.889924045779937im, -3.1129277710557357 + 0.0im, -0.3030303044047327 + 0.0im, -0.01 + 0.0im, -0.001 + 0.0im], [-5.289640837010857 - 8.889924045779928im, -5.289640837010857 + 8.889924045779928im, -3.112927771055735 + 0.0im, -0.3030303044047327 + 0.0im, -0.01 + 0.0im, -0.001 + 0.0im], [-5.289640837010854 - 8.889924045779914im, -5.289640837010854 + 8.889924045779914im, -3.1129277710557295 + 0.0im, -0.3030303044047327 + 0.0im, -0.01 + 0.0im, -0.001 + 0.0im], [-5.289640837010866 - 8.889924045779937im, -5.289640837010866 + 8.889924045779937im, -3.1129277710557433 + 0.0im, -0.3030303044047327 + 0.0im, -0.01 + 0.0im, -0.001 + 0.0im], [-5.289640837010877 - 8.889924045779946im, -5.28964083

In [824]:
fp

6-dimensional StateSpaceSet{Float64} with 6 points
 8.87979  0.481349  0.808949  0.00667582  2.46164e-27  5.91646e-30
 8.87979  0.481349  0.808949  0.00667582  2.46164e-27  5.91646e-30
 8.87979  0.481349  0.808949  0.00667582  2.46164e-27  5.91646e-30
 8.87979  0.481349  0.808949  0.00667582  9.99092e-27  1.26218e-29
 8.87979  0.481349  0.808949  0.00667582  2.46164e-27  5.91646e-30
 8.87979  0.481349  0.808949  0.00667582  9.99092e-27  1.26218e-29

In [825]:
fp[1]

6-element SVector{6, Float64} with indices SOneTo(6):
 8.879793602041058
 0.481349008022014
 0.8089487370604937
 0.006675820870481918
 2.4616404547421673e-27
 5.9164567891575885e-30

In [826]:
fp[2]

6-element SVector{6, Float64} with indices SOneTo(6):
 8.879793602041074
 0.4813490080220141
 0.8089487370604937
 0.006675820870481923
 2.4616404547421673e-27
 5.9164567891575885e-30